In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import statsmodels.api as sm

from sklearn.preprocessing import scale
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics 
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from tabulate import tabulate

# Ruta del archivo a trabajar
res_file_path = 'C:/Users/Toba/FCE/Big Data/Github/BigDataUBA-Grupo28/TP/TP4/Bases/respondieron.xlsx'

# Leer el archivo
df_res = pd.read_excel(res_file_path)

In [2]:
df_res['ESTADO'].unique()

array([3, 1, 2, 4], dtype=int64)

In [3]:
df_res['CH07'].unique()

array([4, 3, 2, 5, 1], dtype=int64)

In [4]:
df_res['CH06'].unique()

array(['79.0', '67.0', '74.0', '54.0', '51.0', '25.0', '20.0', '55.0',
       '48.0', '17.0', '15.0', '77.0', '50.0', '49.0', '26.0', '22.0',
       '18.0', '86.0', '62.0', '76.0', '45.0', '41.0', '21.0', '31.0',
       '19.0', '52.0', '12.0', '32.0', '29.0', '64.0', '10.0', '47.0',
       '43.0', '13.0', '3.0', '46.0', '78.0', '60.0', '59.0', '37.0',
       '44.0', '30.0', '63.0', '71.0', '87.0', '34.0', '14.0', '24.0',
       '28.0', '27.0', '69.0', '56.0', '33.0', '66.0', '61.0', '1.0',
       '92.0', 'Menos de 1 año', '65.0', '57.0', '23.0', '4.0', '40.0',
       '16.0', '9.0', '83.0', '6.0', '38.0', '75.0', '11.0', '53.0',
       '73.0', '39.0', '35.0', '7.0', '5.0', '36.0', '8.0', '82.0',
       '85.0', '72.0', '70.0', '89.0', '42.0', '2.0', '58.0', '68.0',
       '81.0', '80.0', '93.0', '96.0', '84.0', '90.0', '94.0', '88.0',
       '98 y más años', '85', '59', '57', '26', '88', '81', '43', '66',
       '50', '30', '10', '5', '46', '56', '61', '62', '29', '77', '80',
       '78'

In [5]:
df_res['CH04'].unique()

array([1, 2], dtype=int64)

In [6]:
df_res['IPCF'].unique()

array([3.6800e+02, 0.0000e+00, 4.4000e+02, ..., 1.5825e+05, 2.7250e+05,
       9.1000e+05])

In [7]:
df_res['NIVEL_ED'].unique()

array([3, 2, 6, 5, 4, 1, 0, 7], dtype=int64)

In [8]:
# Crear la columna 'DESOCUPADO' (1 si ESTADO es 2, 0 si no lo es)
df_res['DESOCUPADO'] = (df_res['ESTADO'] == 2).astype(int)

In [9]:
#Punto A
años = df_res['ANO4'].unique()

# Inicializar un DataFrame vacío para almacenar la diferencia de medias
diff_means_list = []

for año in años:
    # Filtrar por año
    df_año = df_res[df_res['ANO4'] == año].copy()
    
    # Limpiar CH06 (edad)
    df_año['CH06'] = df_año['CH06'].replace({
        'Menos de 1 año': 0,
        '98 y más años': 98
    }).astype(float)

    # Variables numéricas
    X_num = df_año[['CH06', 'NIVEL_ED', 'IPCF', 'CH04']]

    # Variables categóricas
    X_cat = df_año[['CH07']]  # solo estado civil

    # Transformar categóricas
    X_cat_dummies = pd.get_dummies(X_cat, drop_first=True)

    # Combinar todo
    X_final = pd.concat([X_num, X_cat_dummies], axis=1)

    # Variable dependiente
    y = df_año['DESOCUPADO']
    
    # Dividir en entrenamiento (70%) y prueba (30%)
    X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=444)
    
    # Agregar columna de unos
    X_train = pd.concat([pd.DataFrame({'ones': [1] * X_train.shape[0]}, index=X_train.index), X_train], axis=1)
    X_test = pd.concat([pd.DataFrame({'ones': [1] * X_test.shape[0]}, index=X_test.index), X_test], axis=1)
    
    # Calcular la diferencia de medias
    mean_train = X_train.mean()
    mean_test = X_test.mean()
    mean_diff = mean_train - mean_test
    
    diff_df = pd.DataFrame({
        'Media Entrenamiento': mean_train,
        'Media Prueba': mean_test,
        'Diferencia': mean_diff
    })
    
    diff_means_list.append(diff_df)

# Concatenar resultados de todos los años
diff_means_df = pd.concat(diff_means_list, keys=años)

# Mostrar resultado final
print(diff_means_df)

               Media Entrenamiento   Media Prueba    Diferencia
2004 ones                 1.000000       1.000000      0.000000
     CH06                33.451076      34.050611     -0.599535
     NIVEL_ED             2.691113       2.661867      0.029246
     IPCF               358.227007     373.852669    -15.625662
     CH04                 1.532647       1.520942      0.011705
     CH07                 3.512067       3.456370      0.055697
2024 ones                 1.000000       1.000000      0.000000
     CH06                38.205544      38.105314      0.100231
     NIVEL_ED             3.832892       3.849263     -0.016371
     IPCF            165539.752107  150432.498954  15107.253153
     CH04                 1.518647       1.533999     -0.015352
     CH07                 3.431424       3.436519     -0.005095


In [10]:
# Mapear y limpiar CH06
df_res['CH06'] = df_res['CH06'].replace({
    'Menos de 1 año': 0,
    '98 y más años': 98
}).astype(float)

# Renombrar CH06 a 'edad'
df_res = df_res.rename(columns={'CH06': 'edad'})

# 1. Crear edad2
df_res['edad2'] = df_res['edad'] ** 2

In [11]:
df_res['CH13'].unique()

array(['No', 'Sí', '0.0', 'Ns./Nr.', '1', '2', '0', '9'], dtype=object)

In [12]:
df_res['CH12'].unique()

array(['Secundario', 'Primario', 'Universitario', 'Terciario', '0.0',
       'Jardín/Preescolar', 'Polimodal', 'EGB',
       'Educación especial (discapacitado)', 'Ns./Nr.',
       'Posgrado Universitario', '2', '7', '4', '1', '6', '0', '9', '8',
       '3', '99', '5'], dtype=object)

In [13]:
df_res['CH14'].unique()

array(['03', '  ', '04', '02', '01', '06', '00', '05', '07', '09', '99',
       '08', '98', nan, '2.0', '1.0', '0.0', '3.0', '99.0', '4.0', '5.0',
       '98.0', '6.0', '8.0', '7.0', '9.0'], dtype=object)

In [14]:
# Mapear CH13
map_ch13 = {
    'Sí': 1, '1': 1,
    'No': 2, '2': 2,
    'Ns./Nr.': 3, '0': 3, '0.0': 3, '9': 3
    
}

df_res['CH13'] = df_res['CH13'].replace(map_ch13)
df_res['CH13'] = pd.to_numeric(df_res['CH13'], errors='coerce').fillna(0).astype(int)

# Mapear CH12
map_ch12 = {
    'Jardín/Preescolar': 1, '1': 1,
    'Primario': 2, '2': 2,
    'EGB': 3, '3': 3,
    'Secundario': 4, '4': 4,
    'Polimodal': 5, '5': 5,
    'Terciario': 6, '6': 6,
    'Universitario': 7, '7': 7,
    'Posgrado Universitario': 8, '8': 8,
    'Educación especial (discapacitado)': 9, '9': 9, 
     '0': 0, '0.0': 0, '99': 0
}

df_res['CH12'] = df_res['CH12'].replace(map_ch12)
df_res['CH12'] = pd.to_numeric(df_res['CH12'], errors='coerce').fillna(0).astype(int)

C:\Users\Toba\AppData\Local\Temp\ipykernel_13300\3713707569.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_res['CH13'] = df_res['CH13'].replace(map_ch13)


In [15]:
import numpy as np
import pandas as pd

# Mapear CH13
map_ch13 = {
    'Sí': 1, '1': 1,
    'No': 2, '2': 2,
    'Ns./Nr.': 3, '0': 3, '0.0': 3, '9': 3
}
df_res['CH13'] = df_res['CH13'].replace(map_ch13)
df_res['CH13'] = df_res['CH13'].infer_objects(copy=False)
df_res['CH13'] = pd.to_numeric(df_res['CH13'], errors='coerce').fillna(0).astype(int)

# Mapear CH12
map_ch12 = {
    'Jardín/Preescolar': 1, '1': 1,
    'Primario': 2, '2': 2,
    'EGB': 3, '3': 3,
    'Secundario': 4, '4': 4,
    'Polimodal': 5, '5': 5,
    'Terciario': 6, '6': 6,
    'Universitario': 7, '7': 7,
    'Posgrado Universitario': 8, '8': 8,
    'Educación especial (discapacitado)': 9, '9': 9,
    '0': 0, '0.0': 0, '99': 0
}
df_res['CH12'] = df_res['CH12'].replace(map_ch12)
df_res['CH12'] = pd.to_numeric(df_res['CH12'], errors='coerce').fillna(0).astype(int)

# Asegurar que CH14 también sea numérico
df_res['CH14'] = pd.to_numeric(df_res['CH14'], errors='coerce').fillna(0).astype(int)

# Crear función para calcular años de educación
def calcular_educ_total(row):
    nivel = row['CH12']
    finalizo = row['CH13']
    ch14 = row['CH14']

    # Diccionario de años por nivel (coherente con los códigos)
    años_por_nivel = {
        1: 0,   # Jardín
        2: 7,   # Primario
        3: 9,   # EGB
        4: 12,  # Secundario
        5: 12,  # Polimodal
        6: 15,  # Terciario
        7: 17,  # Universitario
        8: 20,  # Posgrado
        9: 0    # Educación especial (no le asignamos años definidos)
    }

    # Si finalizó, tomar total del nivel
    if finalizo == 1:
        return años_por_nivel.get(nivel, np.nan)
    else:
        años_previos = años_por_nivel.get(nivel - 1, 0) if nivel > 1 else 0
        total = años_previos + ch14
        return min(total, 20)  # máximo 20 años

# Aplicar al DataFrame
df_res['educ'] = df_res.apply(calcular_educ_total, axis=1)

# Filtrar valores válidos
df_res['educ'] = df_res.apply(calcular_educ_total, axis=1)
df_res.loc[(df_res['educ'] <= 0) | (df_res['educ'] >= 98), 'educ'] = np.nan

In [16]:
# Filtrar solo los valores mayores a 0 en 'P21'
df_res = df_res[df_res['P21'] > 0].copy()  # Asegurarse de hacer una copia explícita

# Definir la tasa de inflación
inflacion = 100587.83 / 100  # 100,587.83% de inflación

# Convertir los ingresos de 2004 a 2024 usando la inflación
df_res.loc[:, 'Ingreso_2024'] = np.where(df_res['ANO4'] == 2004,
                                                           df_res['P21'] * (1 + inflacion), 
                                                           df_res['P21'])

# Asegurarse de que los ingresos en 2024 no sean negativos o muy bajos
df_res.loc[:, 'Ingreso_2024'] = df_res['Ingreso_2024'].apply(lambda x: max(x, 0))

# Calcular el salario semanal dividiendo por 40 horas a la semana
df_res.loc[:, 'salario_semanal'] = df_res['Ingreso_2024'] / 4

In [17]:
df_res['CH09'].unique()

array(['Sí', 'No', 'Ns./Nr.', '1', '2'], dtype=object)

In [18]:
df_res['CH11'].unique()

array(['0.0', 'Público', 'Privado', '0', '2', '1', '9'], dtype=object)

In [19]:
# Mapear CH09
map_ch09 = {
    'Sí, asiste': 1, '1': 1,
    'No asiste, pero asistió': 2, '2': 2,
    'Nunca asistió': 0, '3': 0
}

df_res['CH09'] = df_res['CH09'].replace(map_ch09)
df_res['CH09'] = pd.to_numeric(df_res['CH09'], errors='coerce').fillna(0).astype(int)

# Mapear CH11
map_ch11 = {
    '... público': 1, '1': 1,
    '... privado': 2, '2': 2,
    '... Ns/Nr': 9, '9': 9,
    '0': 9, '0.0': 9
}

df_res['CH11'] = df_res['CH11'].replace(map_ch11)
df_res['CH11'] = pd.to_numeric(df_res['CH11'], errors='coerce').fillna(9).astype(int)

In [22]:
# Filtrar solo los ocupados
df_ocupados = df_res[df_res['ESTADO'] == 1]  # Ajusta el nombre de la columna según tu dataset

df_ocupados['mujer'] = np.where(df_ocupados['CH04'] == 2, 1, 0)  # 1 si mujer, 0 si no
df_ocupados = df_ocupados.dropna(subset=['educ'])


# Variables predictoras
X1 = df_ocupados[['edad']]  # Modelo i
X2 = df_ocupados[['edad', 'edad2']]  # Modelo ii
X3 = df_ocupados[['edad', 'edad2', 'educ']]  # Modelo iii
X4 = df_ocupados[['edad', 'edad2', 'educ', 'mujer']]  # Modelo iv
X5 = df_ocupados[['edad', 'edad2', 'educ', 'mujer', 'CH11', 'CH09']]  # Modelo v (sustituye por las variables que creaste)

# Variable dependiente
y = df_ocupados['salario_semanal']

# Añadir la constante para el intercepto
X1 = sm.add_constant(X1)
X2 = sm.add_constant(X2)
X3 = sm.add_constant(X3)
X4 = sm.add_constant(X4)
X5 = sm.add_constant(X5)

# Crear un diccionario para almacenar los resultados
resultados = {}

# Estimar cada modelo
modelos = {'Modelo i': X1, 'Modelo ii': X2, 'Modelo iii': X3, 'Modelo iv': X4, 'Modelo v': X5}

for modelo, X_data in modelos.items():
    # Ajustar el modelo de regresión lineal
    model = sm.OLS(y, X_data).fit()
    
    # Guardar los coeficientes y los desvíos estándar
    coeficientes = model.params.round(3)
    desviacion = model.bse.round(2)
    
    # Almacenar en el diccionario
    resultados[modelo] = pd.DataFrame({'Coeficiente': coeficientes, 'Desvío estándar': desviacion})
    
# Mostrar los resultados
for modelo, resultado in resultados.items():
    print(f"\n{modelo}:\n")
    print(resultado)


Modelo i:

       Coeficiente  Desvío estándar
const    93482.494         10982.67
edad      1235.645           256.60

Modelo ii:

       Coeficiente  Desvío estándar
const   -65696.295         29113.48
edad      9510.613          1425.56
edad2      -96.324            16.32

Modelo iii:

       Coeficiente  Desvío estándar
const  -222709.813         31126.32
edad      9995.236          1403.01
edad2      -95.973            16.06
educ     11125.782           863.33

Modelo iv:

       Coeficiente  Desvío estándar
const  -219735.962         30840.99
edad     10756.519          1392.30
edad2     -104.177            15.94
educ     11992.135           860.03
mujer   -67021.017          6910.69

Modelo v:

       Coeficiente  Desvío estándar
const  -250973.344         34566.31
edad     11330.248          1369.11
edad2     -105.490            15.61
educ     15338.860           868.07
mujer   -62516.812          6747.64
CH11      1845.660          2226.24
CH09   -111000.825          7101.58
